In [2]:
import pandas as pd
import numpy as np
import tqdm
import lightgbm as lgb
from  datetime import datetime, timedelta

### Import dataset

In [3]:
tweets = pd.read_json("/Users/sam/Documents/Masters/Data_Systems_Project/Locations/user_locations.json")

In [4]:
tweets.head()

,created_at,id,text,coordinates,lang,place,user_location
0,2020-05-07 11:56:39+00:00,1258365124517203969,This is a sad reality. AmeriKKKa must realize ...,None,en,None,"Detroit, MI"
1,2020-05-07 11:56:42+00:00,1258365136999518208,RT @JhadiyahG: Please Share Ahmaud Arbery was ...,None,en,None,"Philadelphia, PA"
2,2020-05-07 11:57:00+00:00,1258365210827644930,RT @SharLovegrove: DEVASTATED by the @guardian...,None,en,None,"Crazy Horse, ND"
3,2020-05-07 11:57:05+00:00,1258365231497203714,RT @ZayyWhosane: This shit so frustrating #Bla...,None,en,None,"New Orleans, LA"
4,2020-05-07 11:57:23+00:00,1258365308751970304,We can't break down on a public road with car ...,None,en,"{'id': '7f88808f3d4e1c41', 'url': 'https://api...","Mequon, WI"


In [ ]:
#location
splitted=tweets["user_location"].str.split(", ", n = 1, expand = True)
tweets["state"] = splitted[1]
tweets["city"] = splitted[0]
tweets = tweets.drop(columns=['user_location', 'id', 'text', 'coordinates', 'lang', 'place'])

#date
def split(x):
    x = str(x).split()[0]
    return x
tweets['created_at'] = tweets['created_at'].map(split)
tweets['created_at'] = pd.to_datetime(tweets['created_at'], format="%Y-%m-%d")
tweets = tweets[tweets['created_at']>'2020-05-20']

#

In [17]:
grouped.head()

,created_at,state,city,counts
1,2020-05-26,15,she/her,2
3,2020-05-26,16,she/her,2
7,2020-05-26,17,any pronouns,4
8,2020-05-26,17,he / him,3
10,2020-05-26,"17, 🇯🇲",she/they,5


In [22]:
print(tweets.groupby(['createt_at', 'state', 'city']).size())


In [16]:
grouped = tweets.groupby(['created_at','state', 'city']).size().reset_index(name='counts')
grouped = grouped[grouped.counts>1]

In [ ]:
grouped.head()

In [ ]:
protest = pd.read_csv('ACLEDNeatAdress.csv')
protest['protest'] = 1
protest = protest.rename(columns={"EVENT_DATE": "created_at", "state2": "state", 'place': 'city'})
protest['created_at'] = pd.to_datetime(protest['created_at'], format="%Y-%m-%d")

In [ ]:
tweets["created_at"] = pd.to_datetime(calendar["created_at"])


#transform cat to int
for col, cat in cal_types.items():
    if cat == "category":
        calendar[col] = calendar[col].cat.codes.astype("int16")
        calendar[col] -= calendar[col].min()